# segmenting and clustering neighborhoods in toronto

#### In this third notebook we will explore and visualize neighborhoods in toronto and then segment and cluster neighborhoods in the city of toronto based on borough and neighborhood information

In [1]:
import pandas as pd
import numpy as np
#!conda install -c conda-forge geopy --yes # download goepy package
from geopy.geocoders import Nominatim
# Matplotlib and associated plotting modules
import matplotlib.colors as colors
!conda install -c conda-forge folium=0.5.0 --yes
import folium
import requests # library to handle request
from pandas.io.json import json_normalize #for transform json file into pandas dataframe
# import k-means from clustering stage
from sklearn.cluster import KMeans
print('libraries imported!')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /Users/yashsuthar/opt/anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

  altair             conda-forge/noarch::altair-4.1.0-py_1
  branca             conda-forge/noarch::branca-0.4.2-pyhd8ed1ab_0
  fol

#### load geo_data.csv file created in previous notebook

In [4]:
geo_data=pd.read_csv('geo_data.csv')
geo_data.head()

,Unnamed: 0,postal_code,Borough,Neighborhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.761124,-79.324059
1,1,M4A,North York,Victoria Village,43.732658,-79.311189
2,2,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457
3,3,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150
4,4,M6A,North York,Lawrence Manor,43.722079,-79.437507


In [5]:
geo_data.drop('Unnamed: 0', axis=1, inplace=True)
geo_data.shape

(206, 5)

In [7]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [10]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(geo_data['Latitude'], geo_data['Longitude'], geo_data['Borough'], geo_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### we have decided to work with only boroughs that contain word toronto

In [17]:
toronto_data= geo_data[geo_data['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto_data.head()

,postal_code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457
1,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150
2,M7A,Downtown Toronto,Queen's Park,43.659659,-79.390340
3,M5B,Downtown Toronto,Garden District,43.656500,-79.377114
4,M5B,Downtown Toronto,Ryerson,43.658469,-79.378993


#### now we will explore near by popular venues using foursquare api 

In [18]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

#### we define a function that explore popular near by venues top 100 venues that are in neighborhood within 500 meter first we create get request url and then send get request with the help of foursquare api credentials then store in result variable data in JSON format and then clean json and append in venues_list for repeat this process we will create a loop for all neighborhoods in toronto city

In [23]:
def getNearbyVenues(pos, bor, nei, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for p, b, n, lat, lng in zip(pos, bor, nei, latitudes, longitudes):
        print(n)
            
        # create the API request URL for explore endpoint
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            p,
            b,
            n, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    return(venues_list)

#### Now write the code to run the above function on each neighborhood

In [24]:
venue=getNearbyVenues(pos=toronto_data['postal_code'], bor=toronto_data['Borough'], nei=toronto_data['Neighborhood'], 
                              latitudes=toronto_data['Latitude'], 
                              longitudes=toronto_data['Longitude'])

Regent Park
 Harbourfront
Queen's Park
Garden District
 Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond
 Adelaide
 King
Dufferin
 Dovercourt Village
Harbourfront East
 Union Station
 Toronto Islands
Little Portugal
 Trinity
The Danforth West
 Riverdale
Toronto Dominion Centre
 Design Exchange
Brockton
 Parkdale Village
 Exhibition Place
India Bazaar
 The Beaches West
Commerce Court
 Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
 Forest Hill Road Park
High Park
 The Junction South
North Toronto West
 Lawrence Park
The Annex
 North Midtown
 Yorkville
Parkdale
 Roncesvalles
Davisville
University of Toronto
 Harbord
Runnymede
 Swansea
Moore Park
 Summerhill East
Kensington Market
 Chinatown
 Grange Park
Summerhill West
 Rathnelly
 South Hill
 Forest Hill SE
 Deer Park
CN Tower
 King and Spadina
 Railway Lands
 Harbourfront West
 Bathurst Quay
 South Niagara
 Island airport
Rosedale
St. James Town
 Cabbage

In [31]:
toronto_venues = pd.DataFrame([item for venue_list in venue for item in venue_list])
toronto_venues.columns = ['Postal_code',
                  'Borough',
                  'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue_Latitude', 
                  'Venue_Longitude', 
                  'Venue_Category']

In [32]:
toronto_venues.head()

,Postal_code,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457,Regent Park Aquatic Centre,43.660600,-79.361392,Pool
1,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457,Daniels Spectrum,43.660137,-79.361808,Performing Arts Venue
2,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457,Sumach Espresso,43.658135,-79.359515,Coffee Shop
3,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457,Paintbox Bistro,43.660050,-79.362855,Restaurant
4,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457,Thai To Go,43.663418,-79.360710,Thai Restaurant


In [33]:
toronto_venues.shape

(3691, 9)

In [37]:
toronto_venues.groupby(['Postal_code', 'Borough', 'Neighborhood']).count()

Neighborhood Latitude  \
Postal_code Borough          Neighborhood                               
M4E         East Toronto     The Beaches                           45   
M4K         East Toronto      Riverdale                            32   
                             The Danforth West                     28   
M4L         East Toronto      The Beaches West                     45   
                             India Bazaar                          35   
...                                                               ...   
M6R         West Toronto      Roncesvalles                         38   
                             Parkdale                              41   
M6S         West Toronto      Swansea                               4   
                             Runnymede                             41   
M7A         Downtown Toronto Queen's Park                          76   

                                                Neighborhood Longitude  Venue  \
Postal_code Borough          Neighborhood                                       
M4E         East Toronto     The Beaches                            45     45   
M4K         East Toronto      Riverdale                             32     32   
                             The Danforth West                      28     28   
M4L         East Toronto      The Beaches West                      45     45   
                             India Bazaar                           35     35   
...                                                                ...    ...   
M6R         West Toronto      Roncesvalles                          38     38   
                             Parkdale                               41     41   
M6S         West Toronto      Swansea                                4      4   
                             Runnymede                              41     41   
M7A         Downtown Toronto Queen's Park                           76     76   

                                                Venue_Latitude  \
Postal_code Borough          Neighborhood                        
M4E         East Toronto     The Beaches                    45   
M4K         East Toronto      Riverdale                     32   
                             The Danforth West              28   
M4L         East Toronto      The Beaches West              45   
                             India Bazaar                   35   
...                                                        ...   
M6R         West Toronto      Roncesvalles                  38   
                             Parkdale                       41   
M6S         West Toronto      Swansea                        4   
                             Runnymede                      41   
M7A         Downtown Toronto Queen's Park                   76   

                                                Venue_Longitude  \
Postal_code Borough          Neighborhood                         
M4E         East Toronto     The Beaches                     45   
M4K         East Toronto      Riverdale                      32   
                             The Danforth West               28   
M4L         East Toronto      The Beaches West               45   
                             India Bazaar                    35   
...                                                         ...   
M6R         West Toronto      Roncesvalles                   38   
                             Parkdale                        41   
M6S         West Toronto      Swansea                         4   
                             Runnymede                       41   
M7A         Downtown Toronto Queen's Park                    76   

                                                Venue_Category  
Postal_code Borough          Neighborhood                       
M4E         East Toronto     The Beaches                    45  
M4K         East Toronto      Riverdale                     32  
                             The Danforth West       

In [43]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue_Category'].unique())))

There are 289 uniques categories.


In [45]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue_Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Postal_code'] = toronto_venues['Postal_code']
toronto_onehot['Borough'] = toronto_venues['Borough']
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']


toronto_onehot.columns.get_loc('Neighborhood')

195

In [47]:
toronto_onehot.columns.get_loc('Borough')

290

In [48]:
toronto_onehot.columns.get_loc('Postal_code')

289

In [46]:
toronto_onehot.shape

(3691, 291)

In [50]:
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[289]] +[toronto_onehot.columns[290]] + [toronto_onehot.columns[195]]+ list(toronto_onehot.columns[:195])+list(toronto_onehot.columns[196:289])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Postal_code,Borough,Neighborhood,Accessories Store,African Restaurant,Airport,Airport Service,American Restaurant,Animal Shelter,Antique Shop,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M5A,Downtown Toronto,Regent Park,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5A,Downtown Toronto,Regent Park,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5A,Downtown Toronto,Regent Park,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M5A,Downtown Toronto,Regent Park,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M5A,Downtown Toronto,Regent Park,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
toronto_onehot.shape

(3691, 291)

In [53]:
toronto_grouped = toronto_onehot.groupby(['Postal_code', 'Borough', 'Neighborhood']).mean().reset_index()
toronto_grouped

,Postal_code,Borough,Neighborhood,Accessories Store,African Restaurant,Airport,Airport Service,American Restaurant,Animal Shelter,Antique Shop,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,M4K,East Toronto,Riverdale,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.031250,0.0,0.0,0.156250,0.0,0.0,0.0,0.0,0.0
2,M4K,East Toronto,The Danforth West,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,M4L,East Toronto,The Beaches West,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,M4L,East Toronto,India Bazaar,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.028571,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,M6R,West Toronto,Roncesvalles,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.026316,0.0,0.0,0.0,0.0,0.0
70,M6R,West Toronto,Parkdale,0.02439,0.0,0.0,0.0,0.02439,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
71,M6S,West Toronto,Swansea,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
72,M6S,West Toronto,Runnymede,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.024390,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [54]:
toronto_grouped.shape

(74, 291)

#### define a function to sort venues in descending order

In [55]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [56]:
toronto_grouped.iloc[0, 0:10].index.values

array(['Postal_code', 'Borough', 'Neighborhood', 'Accessories Store',
       'African Restaurant', 'Airport', 'Airport Service',
       'American Restaurant', 'Animal Shelter', 'Antique Shop'],
      dtype=object)

In [57]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postal_code', 'Borough','Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postal_code'] = toronto_grouped['Postal_code']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']
for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 3:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
neighborhoods_venues_sorted.head()

,Postal_code,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Beach,Thai Restaurant,Nail Salon,Bar,Park,Pub,Pizza Place,Breakfast Spot,Japanese Restaurant,Mediterranean Restaurant
1,M4K,East Toronto,Riverdale,Vietnamese Restaurant,Chinese Restaurant,Bakery,Bar,Fast Food Restaurant,Grocery Store,Asian Restaurant,Bridal Shop,Café,French Restaurant
2,M4K,East Toronto,The Danforth West,Grocery Store,Bus Line,Coffee Shop,Bank,Skating Rink,Sushi Restaurant,Fried Chicken Joint,French Restaurant,Caribbean Restaurant,BBQ Joint
3,M4L,East Toronto,The Beaches West,Beach,Thai Restaurant,Nail Salon,Bar,Park,Pub,Pizza Place,Breakfast Spot,Japanese Restaurant,Mediterranean Restaurant
4,M4L,East Toronto,India Bazaar,Indian Restaurant,Grocery Store,Restaurant,Café,Theater,Bar,Gastropub,Pakistani Restaurant,Snack Place,Shopping Plaza


In [58]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(['Postal_code', 'Borough', 'Neighborhood'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 1, 0, 0, 0, 1, 1, 1, 0], dtype=int32)

In [59]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.head()

,Cluster Labels,Postal_code,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,M4E,East Toronto,The Beaches,Beach,Thai Restaurant,Nail Salon,Bar,Park,Pub,Pizza Place,Breakfast Spot,Japanese Restaurant,Mediterranean Restaurant
1,0,M4K,East Toronto,Riverdale,Vietnamese Restaurant,Chinese Restaurant,Bakery,Bar,Fast Food Restaurant,Grocery Store,Asian Restaurant,Bridal Shop,Café,French Restaurant
2,1,M4K,East Toronto,The Danforth West,Grocery Store,Bus Line,Coffee Shop,Bank,Skating Rink,Sushi Restaurant,Fried Chicken Joint,French Restaurant,Caribbean Restaurant,BBQ Joint
3,0,M4L,East Toronto,The Beaches West,Beach,Thai Restaurant,Nail Salon,Bar,Park,Pub,Pizza Place,Breakfast Spot,Japanese Restaurant,Mediterranean Restaurant
4,0,M4L,East Toronto,India Bazaar,Indian Restaurant,Grocery Store,Restaurant,Café,Theater,Bar,Gastropub,Pakistani Restaurant,Snack Place,Shopping Plaza


In [63]:
toronto_merged = toronto_data
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhood"], 1).set_index("Postal_code"), on="postal_code")
toronto_merged.head()

,postal_code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457,1,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Pizza Place,Sporting Goods Shop,Fried Chicken Joint,Chinese Restaurant,Sandwich Place
0,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457,1,Coffee Shop,Restaurant,Thai Restaurant,Pharmacy,Auto Dealership,Pool,Pub,Electronics Store,Fast Food Restaurant,Park
1,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150,1,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Pizza Place,Sporting Goods Shop,Fried Chicken Joint,Chinese Restaurant,Sandwich Place
1,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150,1,Coffee Shop,Restaurant,Thai Restaurant,Pharmacy,Auto Dealership,Pool,Pub,Electronics Store,Fast Food Restaurant,Park
2,M7A,Downtown Toronto,Queen's Park,43.659659,-79.390340,1,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Pharmacy,Japanese Restaurant,Bank,Thai Restaurant,French Restaurant,Bubble Tea Shop


In [85]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
rainbow = ['red', 'blue', 'black', 'orange', 'darkgreen']

# add markers to the map
markers_colors = []
for lat, lon, pos, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['postal_code'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{},{},{} Cluster-{}'.format(poi, pos, bor, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [73]:
set(kmeans.labels_)

{0, 1, 2, 3, 4}

In [76]:
first=toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
first

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Downtown Toronto,0,Clothing Store,Coffee Shop,Hotel,Restaurant,Lingerie Store,Theater,Movie Theater,Fast Food Restaurant,Café,Japanese Restaurant
4,Downtown Toronto,0,Clothing Store,Coffee Shop,Hotel,Restaurant,Lingerie Store,Theater,Movie Theater,Fast Food Restaurant,Café,Japanese Restaurant
6,East Toronto,0,Beach,Thai Restaurant,Nail Salon,Bar,Park,Pub,Pizza Place,Breakfast Spot,Japanese Restaurant,Mediterranean Restaurant
7,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Italian Restaurant,Seafood Restaurant,Hotel,Japanese Restaurant,Gastropub,Breakfast Spot,Beer Bar
8,Downtown Toronto,0,Coffee Shop,Clothing Store,Café,Restaurant,Hotel,Bakery,Gastropub,Diner,Plaza,Salad Place
...,...,...,...,...,...,...,...,...,...,...,...,...
67,Downtown Toronto,0,Coffee Shop,Restaurant,Italian Restaurant,Gym,Yoga Studio,Bar,Sandwich Place,Café,Beer Bar,Pizza Place
67,Downtown Toronto,0,Yoga Studio,Pizza Place,Dessert Shop,Bakery,Middle Eastern Restaurant,Optical Shop,Café,Spa,Ramen Restaurant,Salad Place
67,Downtown Toronto,0,Hotel,Coffee Shop,Pizza Place,Bar,Gym,Baseball Stadium,Scenic Lookout,Aquarium,Ice Cream Shop,Yoga Studio
71,Downtown Toronto,0,Coffee Shop,Hotel,Restaurant,Café,Gym,Japanese Restaurant,Salad Place,Steakhouse,American Restaurant,Deli / Bodega


In [77]:
second=toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
second

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Pizza Place,Sporting Goods Shop,Fried Chicken Joint,Chinese Restaurant,Sandwich Place
0,Downtown Toronto,1,Coffee Shop,Restaurant,Thai Restaurant,Pharmacy,Auto Dealership,Pool,Pub,Electronics Store,Fast Food Restaurant,Park
1,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Pizza Place,Sporting Goods Shop,Fried Chicken Joint,Chinese Restaurant,Sandwich Place
1,Downtown Toronto,1,Coffee Shop,Restaurant,Thai Restaurant,Pharmacy,Auto Dealership,Pool,Pub,Electronics Store,Fast Food Restaurant,Park
2,Downtown Toronto,1,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Pharmacy,Japanese Restaurant,Bank,Thai Restaurant,French Restaurant,Bubble Tea Shop
...,...,...,...,...,...,...,...,...,...,...,...,...
70,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Indian Restaurant,Beer Store,Japanese Restaurant,Italian Restaurant,Pizza Place,Diner,Gastropub
70,Downtown Toronto,1,Coffee Shop,Café,Pizza Place,Grocery Store,Bakery,Market,Caribbean Restaurant,Breakfast Spot,Library,Bank
71,Downtown Toronto,1,Breakfast Spot,Bank,Pizza Place,Outdoor Supply Store,Chinese Restaurant,Sandwich Place,Fast Food Restaurant,Fish Market,Sporting Goods Shop,Gas Station
72,Downtown Toronto,1,Breakfast Spot,Bank,Pizza Place,Outdoor Supply Store,Chinese Restaurant,Sandwich Place,Fast Food Restaurant,Fish Market,Sporting Goods Shop,Gas Station


In [78]:
third=toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
third

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,West Toronto,2,Pilates Studio,Park,Bus Line,Dance Studio,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Filipino Restaurant,Fish & Chips Shop,Doner Restaurant
50,West Toronto,2,Pilates Studio,Park,Bus Line,Dance Studio,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Filipino Restaurant,Fish & Chips Shop,Doner Restaurant
56,Central Toronto,2,Bank,Home Service,Playground,Park,Ethiopian Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant
57,Central Toronto,2,Bank,Home Service,Playground,Park,Ethiopian Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant
58,Central Toronto,2,Bank,Home Service,Playground,Park,Ethiopian Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant
59,Central Toronto,2,Bank,Home Service,Playground,Park,Ethiopian Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant
60,Central Toronto,2,Bank,Home Service,Playground,Park,Ethiopian Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant
68,Downtown Toronto,2,Park,Playground,Bike Trail,Yoga Studio,Factory,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant


In [79]:
forth=toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
forth

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,Central Toronto,3,Skating Rink,Athletics & Sports,Gym / Fitness Center,Yoga Studio,Ethiopian Restaurant,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store
36,Central Toronto,3,Skating Rink,Athletics & Sports,Gym / Fitness Center,Yoga Studio,Ethiopian Restaurant,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store


In [80]:
five=toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
five

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Downtown Toronto,4,Harbor / Marina,Music Venue,Café,Park,Yoga Studio,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant
16,Downtown Toronto,4,Harbor / Marina,Music Venue,Café,Park,Yoga Studio,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant
17,Downtown Toronto,4,Harbor / Marina,Music Venue,Café,Park,Yoga Studio,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant


#### clothing store, Restaurant, hotels falls in cluster one and frequency of venues categrories is very high in cluster one
#### coffe shops, cafe, pizza place, Restaurant(ethiopian, fast food) falls in cluster two and frequency of venues categrories is very high in cluster one
#### bank, playground, park, home service falls in cluster three and frequency of venue categrories is high in cluster three
#### skating rink, athletics&sports, gym, fitness center falls in cluster four and frequency of venue categrories is low in cluster four
#### harbor/marina, music venue falls in cluster five and frequency of venue categrories is low in cluster five
#### business areas falls in cluster one and cluster two